In [ ]:
import gurobipy as gp
from rsnn.optim import QProgram
import numpy as np

In [22]:
K = 10000
N = 500

A = np.random.randn(N, K)

# b = A @ np.random.randn(K) + np.random.rand(N) * 1e-6 # Adding noise to the left-hand side, make sure the problem is feasible
b = np.random.randn(N) 

In [23]:
# Solve the following MIP:
#  minimize
#        s
#  subject to
#        A x <= b + s


# Create a new model
m = gp.Model()

# Create variables
x = m.addMVar(shape=K, name="x")
s = m.addVar(name="s")

# Add constraints
m.addConstr(A @ x - s <= b , name="constraints")

# Set objective function
m.setObjective(s, gp.GRB.MINIMIZE)

# # Add constraints
# m.addConstr(x + 2 * y + 3 * z <= 4)
# m.addConstr(x + y >= 1)

# Solve it!
m.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[rosetta2] - Darwin 24.5.0 24F74)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 500 rows, 10001 columns and 5000500 nonzeros
Model fingerprint: 0xffca2915
Coefficient statistics:
  Matrix range     [3e-07, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-04, 3e+00]
Presolve time: 0.79s
Presolved: 500 rows, 10001 columns, 5000500 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.248e+05
 Factor NZ  : 1.252e+05 (roughly 5 MB of memory)
 Factor Ops : 4.179e+07 (less than 1 second per iteration)
 Threads    : 6

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.41005731e-01  0.00000000e+00  4.07e+02 0.00e+00  1.06e+00  

In [ ]:
solver = QProgram(K)
solver.add_constraints(A, b)
solver.solve(atol=1e-12)

2025-07-03 13:40:30.288' - INFO - Solver initialized with 10000 variables and 0 constraints.
2025-07-03 13:40:30.403' - DEBUG - 500 new constraint(s) successfully added. The problem now has 500 constraints.
2025-07-03 13:40:30.584' - DEBUG - DBFFD - Iteration 1:
	dual cost is 0.013684551346797042 and primal cost is 0.013798757795865757
2025-07-03 13:40:30.775' - DEBUG - DBFFD - Iteration 2:
	dual cost is 0.013867377442742626 and primal cost is 0.013865471398367846
2025-07-03 13:40:30.972' - DEBUG - DBFFD - Iteration 3:
	dual cost is 0.013867460003902796 and primal cost is 0.0138674848980683
2025-07-03 13:40:30.973' - INFO - DBFFD - Feasible solution found in 3 iteration(s) with cost 0.0138674848980683.


1

In [25]:
# Solve the following MIP:
#  minimize
#        x^T x / 2
#  subject to
#        A x <= b


# Create a new model
m = gp.Model()

# Create variables
x = m.addMVar(shape=K, name="x")

# Add constraints
m.addConstr(A @ x <= b , name="constraints")

# Set objective function
m.setObjective(x @ x, gp.GRB.MINIMIZE)

# Solve it!
m.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[rosetta2] - Darwin 24.5.0 24F74)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 500 rows, 10000 columns and 5000000 nonzeros
Model fingerprint: 0xda5f8ad2
Model has 10000 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-07, 5e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-04, 3e+00]
Presolve time: 0.51s
Presolved: 500 rows, 10000 columns, 5000000 nonzeros
Presolved model has 10000 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.248e+05
 Factor NZ  : 1.252e+05 (roughly 5 MB of memory)
 Factor Ops : 4.179e+07 (less than 1 second per iteration)
 Threads    : 8

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.49999972e+09 -2.49999972e+09

In [26]:
gb_x = np.array(m.x)
solver.x.T @ solver.x, gb_x.T @ gb_x

(np.float64(0.0277349697961366), np.float64(0.05901623936359704))

In [15]:
np.all(A @ solver.x - b <= 1e-6), np.all(A @ gb_x - b <= 1e-6)

(np.True_, np.True_)

In [ ]:
sigma2 = 1e9
e1 = np.array([1.0] + [0.0] * K)
solver = QProgram(K+1, mxf=-sigma2 * e1, Vxf=sigma2 * np.eye(K+1), beta_init=1e-18)
solver.add_constraints(np.block([[-np.ones((N,1)), A]]), b)
solver.solve(n_iter=1000)
solver.x[0]

2025-07-03 13:17:41.851' - DEBUG - 50 new constraint(s) successfully added. The problem now has 50 constraints.
2025-07-03 13:17:41.855' - DEBUG - DBFFD - Iteration 1:
	dual cost is -0.19536895296710458 and primal cost is -339943917.41606945
2025-07-03 13:17:41.858' - DEBUG - DBFFD - Iteration 2:
	dual cost is -0.2360858244368538 and primal cost is -264315172.15486157
2025-07-03 13:17:41.863' - DEBUG - DBFFD - Iteration 3:
	dual cost is -0.23797858530803012 and primal cost is -261777241.73849916
2025-07-03 13:17:41.866' - DEBUG - DBFFD - Iteration 4:
	dual cost is -0.23798054733372737 and primal cost is -262019452.66627264
2025-07-03 13:17:41.870' - DEBUG - DBFFD - Iteration 5:
	dual cost is -0.23798054733372742 and primal cost is -262019452.66627264
2025-07-03 13:17:41.875' - DEBUG - DBFFD - Iteration 6:
	dual cost is -0.23798054733372737 and primal cost is -262019452.66627264
2025-07-03 13:17:41.878' - DEBUG - DBFFD - Iteration 7:
	dual cost is -0.23798054733372742 and primal cost is

np.float64(-575649748.1355557)

In [ ]:
# # Solve the following MIP:
# #  minimize
# #        0
# #  subject to
# #        A x <= b

# # Create a new model
# m = gp.Model()

# # Create variables
# x = m.addMVar(shape=K, name="x")

# # Add constraints
# m.addConstr(A @ x <= b , name="constraints")

# # Solve it!
# m.optimize()

# print(f"Optimal objective value: {m.objVal}")

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[rosetta2] - Darwin 24.5.0 24F74)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 50 rows, 10000 columns and 500000 nonzeros
Model fingerprint: 0x6ac0e03c
Coefficient statistics:
  Matrix range     [3e-06, 5e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e-03, 2e+00]
Presolve time: 0.07s
Presolved: 50 rows, 10000 columns, 500000 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.225e+03
 Factor NZ  : 1.275e+03 (roughly 4 MB of memory)
 Factor Ops : 4.292e+04 (less than 1 second per iteration)
 Threads    : 6

Barrier performed 0 iterations in 0.13 seconds (0.12 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with primal simplex
Iteration    Objective       Primal Inf.  